In [ ]:
from main import main

In [1]:
!python main.py assets/sample.txt "text test"

Extracting text from: assets/sample.txt
Text extracted: Andre Achtar-Zadeh is a software engineer at OpenAI. He works on natural language models and AI systems.
 ...
Initializing Qdrant and inserting vector...
Running entity and relationship extraction...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
LLM response:
{
  "entities": ["Andre Achtar-Zadeh", "software engineer", "OpenAI", "natural lan

In [2]:
!python main.py assets/sample_image.png "png test"

Extracting text from: assets/sample_image.png
Text extracted: chrom,
hollow”
 ...
Initializing Qdrant and inserting vector...
Running entity and relationship extraction...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
LLM response:
{
  "entities": ["chrom", "hollow"],
  "relationships": []
}
Parsing extracted JSON data...
Inserting entities and relationships into Neo4j...
Running similarity sear

In [3]:
!python main.py assets/sample_pdf.pdf "pdf test"

Extracting text from: assets/sample_pdf.pdf
Text extracted:  
72-Hour Technical Challenge 
Multimodal Enterprise RAG – Leveraging Knowledge Graphs and Hybrid Search 
 
Objective 
Design and implement a modular prototype of an Enterprise Retrieval-Augmented 
Generation (RAG) system that supports text, image, audio, and video ingestion, builds a 
searchable knowledge graph, and enables hybrid search using, keyword and vector 
retrieval (Graph RAG System). 
You are expected to begin with evals to define success criteria and to structure your 
architecture a ...
Initializing Qdrant and inserting vector...
Running entity and relationship extraction...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizer

In [4]:
!python main.py assets/sample_audio.mp3 "mp3 test"

Extracting text from: assets/sample_audio.mp3
Text extracted:  It's a weekend. In John decided to watch the latest movie recommended by Netflix at his friend's place. Before heading out, he asked Siri about the weather and realized it would rain. So he decided to take his Tesla for the long journey and switch to autopilot on the highway. After coming home from the eventful day, he started wondering how technology has made his life easy. He did some research on the internet and found out that Netflix, Siri and Tesla are all using AI. So what is AI? AI or ar ...
Initializing Qdrant and inserting vector...
Running entity and relationship extraction...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokeniz

In [5]:
!python main.py assets/Aiinfo.mp4 "mp4 test"

Extracting text from: assets/Aiinfo.mp4
MoviePy - Writing audio in temp_audio.wav
MoviePy - Done.                                                                 
Text extracted:  Everybody's talking about artificial intelligence these days, AI. Machine learning is another hot topic. Are they the same thing or are they different? And if so, what are those differences? And deep learning is another one that comes into play. I actually did a video on these three, artificial intelligence, machine learning and deep learning, and talked about where they fit. And there were a lot of comments on that, and I read those comments, and I'd like to address some of the most frequently ...
Initializing Qdrant and inserting vector...
Running entity and relationship extraction...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the 

In [6]:
import json

with open("results/hybrid_results.json") as f:
    results = json.load(f)

for r in results:
    print("Result:\n", r, "\n---")

Result:
 vector_results 
---
Result:
 graph_results 
---
